<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<h2 style="text-align: center;"><b>Основы фреймворка TensorFlow</b></h2>

<img src="https://cdn-images-1.medium.com/max/1600/1*eFRgat2Iy6wZpi_DEItKgA.png" width=300 height=225>

<h4 style="text-align: center;"><b>Составитель: Илья Захаркин (ФИВТ МФТИ, NeurusLab). По всем вопросам в Telegram: <a>@ilyazakharkin</a></b></h4>

В этом ноутбуке мы более близко познакомимся с TF и попрактикуемся в классификации на FashionMNIST с помощью этого фреймворка. Пайплайн у него очень похож на PyTorch и Keras, однако есть много технических нюансов, которым и посвящён этот ноутбук.

*Примечание*: рекомендуется выполнять этот ноутбук в Google Colab во избежание проблем с установкой (если ставить `tensorflow`  без GPU, то проблем, на самом деле, быть не должно).

<h3 style="text-align: center;"><b>Основы: граф и сессия</b></h3>

Если tf не установлен:

In [0]:
# !pip install tensorflow

Импортируем:

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

В TF есть две основные сущности -- это ***граф вычислений `tf.Graph()`*** и ***сессия `tf.Session()`***.

Сначала *строится* граф вычислений, а потом в сессии он *исполняется*.

Пример графа вычислений (очень условный):

<img src="https://camo.qiitausercontent.com/137bc298bf30fe06e61b59a638fea966d272f2b8/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f313034362f38663430386438392d666563362d643761632d633961662d3865393061383039613233622e706e67" width=300 height=400>

А вот пример реального графа (скрин из TensorBoard):

<img src="https://blog.altoros.com/wp-content/uploads/2016/05/visualizing-graphs-with-tensorboard-wxb-group.png">

То есть граф -- это нечто построенное нами до сессии, а сессия -- это то, во время чего все операции в графе исполняются (при подаче в него входных данных).

Помимо этого введения, есть много замечательных курсов по этому фреймворку, например:

* Начать можно с официального intro: https://www.tensorflow.org/guide/low_level_intro 
* Продолжить этими замечательными практикумами: https://github.com/GoogleCloudPlatform/tensorflow-without-a-phd
* Посмотреть что-то здесь, если не очень понятно: https://github.com/Hvass-Labs/TensorFlow-Tutorials
* Закончить продвинутым курсом: https://github.com/sjchoi86/advanced-tensorflow

И не забудьте заглянуть в официальные туториалы: https://www.tensorflow.org/tutorials/

<h3 style="text-align: center;"><b>Основные объекты</b></h3>

Используя https://www.tensorflow.org/guide/low_level_intro

#### Тензоры (Tensor)

Центральный объект -- это **тензоры `tf.Tensor`**. Это те же `np.array` из `numpy`, только обёрнуты в специальный класс `tf.Tensor`.

Давайте сразу посмотрим на простой пример объявления двух констнатныз тензоров и взятия от них матричного произведения:

In [0]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)

Посомтрим на типы того, чтоы мы создали:

In [0]:
c

In [0]:
d

In [0]:
e

Обратите внимание: выводится тип объекта, его имя, shape и тип элементов, которые лежат внутри тензора.

Как видим, никакой матрицы-результата мы не получили. Это потому, что мы лишь объявлии граф вычислений, но сами вычисления ещё не зпустили. Для этого нужно объявить сессию и исполнить в ней этот граф:

In [0]:
sess = tf.Session()

result = sess.run(e)

In [0]:
result

In [0]:
type(result)

Видим, что при выполнении графа произведение посчиталось и результат положился в виде `np.array` в переменную `result`.

Заметьте: мы не объявляли в начале отедльный граф, поэтмоу текущий граф вычислений можно оплучить так: `tf.get_default_graph()`

In [0]:
tf.get_default_graph()

Давайте визуализируем этот граф с помощью TensorBoard **(сработает только на локальной машине, не на Colab)**:

In [0]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

In [0]:
# pip install tensorboard
# tensorboard --logdir .

Чтобы посмотреть на результат TensorBoard, перейдите в любом браузере по адресу `http://localhost:6006/#graphs` (работает только на локальной машине, не на Colab (однако есть способ, попробуйте разобраться самостоятельно: https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/)

<img src="https://www.tensorflow.org/images/getting_started_add.png">

Ещё пример:

In [0]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

In [0]:
sess = tf.Session()
print(sess.run(total))

Поучительный прмиер, говорящий о том, что при генерации случайных чисел и запуске вычисления графа, вычисления одного тензора происходят **один раз на запуск**, то есть:

In [0]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

Обратите внимание, что у out1 и out2 был один и тот же тензор vec на входе.

#### Плейсхолдеры (Placeholder)

Для того чтобы иметь возможность подавать на вход графу какие-то свои тензоры, используют **`tf.Placeholder()`**.

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [0]:
x

In [0]:
y

In [0]:
z

Это всё те же тензоры в графе вычислений, но они не имеют значений -- их нужно **"скормить" ("feed")** самому:

In [0]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))

== *"вычисли значения тензора z, если на вход подаются x=3 и y=4.5"*

In [0]:
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

== *"вычисли значения тензора z, если на вход подаются x=[1, 3] и y=[2, 4]"* 

#### Переменные (Variable)

Тензоры существуют только во время выполнения сессии, чтобы получить результат вычислений. Для того, чтобы сохранять значения тензоров, в TF есть **`tf.Variable()`**.

`tf.Variable` хранит в себе `tf.Tensor`, однако можно читать и менять его значения в течение нескольких сессий, то есть `Variable` ближе к питоновскому понятию переменной, которая хранится в оперативной памяти компьютера.

Создать переменную можно так:

In [0]:
my_variable = tf.get_variable("my_variable", [1, 2, 3])
my_variable

In [0]:
my_variable.graph

Сейчас она тоже прикреплена к графу по-умолчанию:

In [0]:
tf.get_default_graph()

(такой вывод можно сделать, потому что адреса графа в памяти совпадают)

Создадим ещё одну и инициализируем константной:

In [0]:
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
                                 initializer=tf.constant([23, 42]))

Запустим сессию, чтобы посомтреть на значения переменных (сейчас должна быть ошибка):

In [0]:
sess.run(my_variable)
sess.run(other_variable)

Ничего не вышло, потому что сначала переменные надо инициализировать:

In [0]:
sess.run(tf.global_variables_initializer())

Теперь должно получиться:

In [0]:
sess.run(my_variable)

In [0]:
sess.run(other_variable)

Всё верно: вторая переменная ининцилизировалась своим значеним, заданным в конструкторе, а первая -- случайным "мусором".

<h3 style="text-align: center;"><b>К практике</b></h3>

Напишем свой нейрон на tensorflow. 

Ниже пример регрессии по 4 точкам с помощью одного нейрона 

In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

Нарисуем:

In [0]:
import matplotlib.pyplot as plt

In [0]:
x_values, y_values = sess.run([x, y_true])

plt.scatter(x_values, y_values, c='r');
plt.plot(x_values, y_values);

plt.xticks(x_values)
plt.yticks(y_values)

plt.title('Simple regression', fontsize=15)
plt.xlabel('$x$', fontsize=15)
plt.ylabel('$y$', fontsize=15);

Объявим линейную регрессию:

In [0]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [0]:
linear_model

In [0]:
y_pred

Функция потерь -- среднеквадратичная, оптимизатор -- градиентный спуск:

In [0]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

Иницилизируем переменные, обЪявим сессию и запусим в ней обучение:

In [0]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print('loss: ', loss_value)

print(sess.run(y_pred))

In [0]:
x_values, y_values = sess.run([x, y_pred])

plt.scatter(x_values, y_values, c='r');
plt.plot(x_values, y_values);

plt.xticks(x_values)
plt.yticks(y_values)

plt.title('Prediction of one neuron', fontsize=15)
plt.xlabel('$x$', fontsize=15)
plt.ylabel('$y$ predicted', fontsize=15);

<h3 style="text-align: center;"><b>FashionMNIST (5 баллов)</b></h3>

Для обучения нейросетей на TensorFlow обычно используют либо модуль `.keras` (который, по сути, является Keras'ом, только обращение к нему происходит через TF), либо модуль `tf.Slim`. В этом задании Вам нужно самостоятельно написть обучение нейросети на датасете FashionMNIST и добиться качества (поклассового `accuracy`) **больше, чем 0.9**.

Бейзланом к этому заданию является официальный туториал TensorFlow: https://www.tensorflow.org/tutorials/keras/basic_classification

*Hint*: можно использовать свёрточные слои и слои не из туториала

In [0]:
<Ваш крутой код здесь>

In [0]:
...

<h3 style="text-align: center;"><b>На выбор: Style Transfer или CIFAR10 (каждое по 5 баллов)</b></h3>

В этой части Вам нужно самостоятельно реализовать перенос стиля на TensorFlow или написать классификацию датасета CIFAR10. Можно сделать и то, и то, тогда вы получите 10 баллов, а не 5, и за всё задание будет 15, а не 10.

#### Neural Style Transfer:
Бейзлайн по переносу стиля: https://github.com/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb
Это уже почти решение, надо только написать это сюда и объяснить каждый шаг, и показать пример работы на своих картинках.

In [0]:
<Ваш код высочайшего сорта здесь>

In [0]:
...

#### CIFAR10:
Бейзлайн по классификации CIFAR10: https://www.tensorflow.org/tutorials/images/deep_cnn  
Там есть ссылки на файлы с кодом, который тоже можно прямо сюда вставлять, главное, чтобы качество в итоге получилось хорошим (формально порог не задан, будет смотреться у всех индивиуально).

In [0]:
<Покажите класс>

In [0]:
...